In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

In [2]:
spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/27 23:21:15 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/27 23:21:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 23:21:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/27 23:21:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/27 23:21:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark_Students = spark.read.option('header', True).option('delimiter', ',').csv('Students.csv')
spark_Subjects = spark.read.option('header', True).option('delimiter', ',').csv('Subjects.csv')
spark_Examinations = spark.read.option('header', True).option('delimiter', ',').csv('Examinations.csv')

spark_Students.show(100)
spark_Subjects.show(100)
spark_Examinations.show(100)

+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|       Alice|
|         2|         Bob|
|        13|        John|
|         6|        Alex|
+----------+------------+

+------------+
|subject_name|
+------------+
|        Math|
|     Physics|
| Programming|
+------------+

+----------+------------+
|student_id|subject_name|
+----------+------------+
|         1|        Math|
|         1|     Physics|
|         1| Programming|
|         2| Programming|
|         1|     Physics|
|         1|        Math|
|        13|        Math|
|        13| Programming|
|        13|     Physics|
|         2|        Math|
|         1|        Math|
+----------+------------+



In [4]:
pandas_Students = pd.read_csv('Students.csv')
pandas_Subjects = pd.read_csv('Subjects.csv')
pandas_Examinations = pd.read_csv('Examinations.csv')

In [5]:
pandas_Students.head(100)

,student_id,student_name
0,1,Alice
1,2,Bob
2,13,John
3,6,Alex


In [6]:
pandas_Subjects.head(100)

,subject_name
0,Math
1,Physics
2,Programming


In [7]:
pandas_Examinations.head(100)

,student_id,subject_name
0,1,Math
1,1,Physics
2,1,Programming
3,2,Programming
4,1,Physics
5,1,Math
6,13,Math
7,13,Programming
8,13,Physics
9,2,Math


### SQL

In [8]:
def students_and_examinations(students: pyspark.sql.dataframe.DataFrame, 
                              subjects: pyspark.sql.dataframe.DataFrame, 
                              examinations: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    # Register the 'students' DataFrame as a temporary SQL table/view
    students.createOrReplaceTempView('Students')
    
    # Register the 'subjects' DataFrame as a temporary SQL table/view
    subjects.createOrReplaceTempView('Subjects')
    
    # Register the 'examinations' DataFrame as a temporary SQL table/view
    examinations.createOrReplaceTempView('Examinations')
    
    # Write the SQL query to calculate the number of attended exams for each student and subject
    sqlQuery = \
    '''
    SELECT
        -- Select the student ID from the 'Students' table
        st.student_id, 
        
        -- Select the student name from the 'Students' table
        st.student_name, 
        
        -- Select the subject name from the 'Subjects' table
        sub.subject_name, 
        
        -- Count the number of exams attended by the student for the subject
        -- If no exam attended, return 0 using COALESCE to handle NULLs
        COALESCE(count(ex.student_id), 0) AS attended_exams
        
    FROM Students st 
    
    -- Perform a CROSS JOIN between 'Students' and 'Subjects'
    -- This creates all combinations of students and subjects
    CROSS JOIN 
    Subjects sub
    
    -- Perform a LEFT JOIN between the cross joined students and subjects
    -- with the 'Examinations' table to count the attended exams
    LEFT JOIN
    Examinations ex
    ON 
        -- Join condition: Match student IDs and subject names
        st.student_id = ex.student_id AND sub.subject_name = ex.subject_name
        
    -- Group the result by student ID, student name, and subject name
    GROUP BY st.student_id, st.student_name, sub.subject_name
    
    -- Order the results by student ID, student name, and subject name for easy readability
    ORDER BY st.student_id, st.student_name, sub.subject_name
    '''
    
    # Execute the SQL query and store the result in a DataFrame
    students_With_Count_Subject_Attended_Exams = spark.sql(sqlQuery = sqlQuery)
    
    # Return the result DataFrame
    return students_With_Count_Subject_Attended_Exams
    

output = students_and_examinations(students = spark_Students,
                                   subjects = spark_Subjects,
                                   examinations = spark_Examinations)
output.show(100)

+----------+------------+------------+--------------+
|student_id|student_name|subject_name|attended_exams|
+----------+------------+------------+--------------+
|         1|       Alice|        Math|             3|
|         1|       Alice|     Physics|             2|
|         1|       Alice| Programming|             1|
|        13|        John|        Math|             1|
|        13|        John|     Physics|             1|
|        13|        John| Programming|             1|
|         2|         Bob|        Math|             1|
|         2|         Bob|     Physics|             0|
|         2|         Bob| Programming|             1|
|         6|        Alex|        Math|             0|
|         6|        Alex|     Physics|             0|
|         6|        Alex| Programming|             0|
+----------+------------+------------+--------------+



### PySpark

In [9]:
def students_and_examinations(students: pyspark.sql.dataframe.DataFrame, 
                              subjects: pyspark.sql.dataframe.DataFrame, 
                              examinations: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    
    # Perform a CROSS JOIN between 'students' and 'subjects'
    # This creates all combinations of students and subjects.
    cross_Join = students.join(subjects, how='cross')
    
    # Perform a LEFT JOIN between the result of the cross join and the 'examinations' DataFrame
    # This ensures that we keep all rows from the cross join, and we add examination data where available
    left_Join = cross_Join.join(examinations, on=['student_id', 'subject_name'], how='left')
    
    # Group the data by 'student_id', 'student_name', and 'subject_name' 
    # to count how many exams each student has attended
    # Use count to count the number of occurrences of student_id (which tells us the number of attended exams)
    # In case there are no exams for a student in a subject, the count will be 0
    students_With_Count_Subject_Attended_Exams = left_Join.groupBy(['student_id', 'student_name', 
                                                                    'subject_name'])\
                                                    .agg(count(examinations.student_id)
                                                            .alias('attended_exams'))\
                                                    .orderBy(
                                                        # Order the grouped results by student_id,
                                                        # student_name, and subject_name for readability
                                                        ['student_id', 'student_name', 'subject_name'])\
                                                    .select(
                                                            # Select the relevant columns: 
                                                            # student_id, student_name, subject_name, 
                                                            # and the count of attended exams
                                                            ['student_id', 'student_name', 
                                                             'subject_name', 'attended_exams'])
    
    # Return the resulting DataFrame containing students, subjects, and the count of attended exams
    return students_With_Count_Subject_Attended_Exams

output = students_and_examinations(students = spark_Students,
                                   subjects = spark_Subjects,
                                   examinations = spark_Examinations)
output.show(100)

+----------+------------+------------+--------------+
|student_id|student_name|subject_name|attended_exams|
+----------+------------+------------+--------------+
|         1|       Alice|        Math|             3|
|         1|       Alice|     Physics|             2|
|         1|       Alice| Programming|             1|
|        13|        John|        Math|             1|
|        13|        John|     Physics|             1|
|        13|        John| Programming|             1|
|         2|         Bob|        Math|             1|
|         2|         Bob|     Physics|             0|
|         2|         Bob| Programming|             1|
|         6|        Alex|        Math|             0|
|         6|        Alex|     Physics|             0|
|         6|        Alex| Programming|             0|
+----------+------------+------------+--------------+



### Pandas

In [10]:
def students_and_examinations(students: pd.DataFrame, 
                              subjects: pd.DataFrame, 
                              examinations: pd.DataFrame) -> pd.DataFrame:
    
    # Step 1: Group the 'examinations' DataFrame by 'student_id' and 'subject_name'
    # We count how many exams each student attended for each subject
    # This will give us the number of attended exams for each student-subject pair
    grouped = examinations.groupby(['student_id','subject_name']).size().reset_index(name = 'attended_exams')
    
    # Step 2: Create all possible combinations of students and subjects using a cross join
    # This combines every student with every subject, so we can later match them to the examinations data
    all_id_subjects = pd.merge(students, subjects, how = 'cross')
    
    # Step 3: Perform a LEFT JOIN to merge the all_id_subjects DataFrame with the grouped exams data
    # This step will add the attended exams count to the student-subject pairs
    # If a student did not attend any exams for a subject, the 'attended_exams' will be NaN
    students_With_Count_Subject_Attended_Exams = pd.merge(all_id_subjects, grouped, 
                                                          on = ['student_id', 'subject_name'], 
                                                          how = 'left')
    
    # Step 4: Replace all NaN values (which mean no exams attended) with 0
    # This means that if a student has no exam data for a subject, we'll show 0 instead of NaN
    students_With_Count_Subject_Attended_Exams.fillna(0, inplace = True)
    
    # Step 5: Return the final DataFrame, which includes the student, subject, and attended exam count
    return students_With_Count_Subject_Attended_Exams

output = students_and_examinations(students = pandas_Students, 
                                   subjects = pandas_Subjects,
                                   examinations = pandas_Examinations)
output.head(100)

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,0.0
5,2,Bob,Programming,1.0
6,13,John,Math,1.0
7,13,John,Physics,1.0
8,13,John,Programming,1.0
9,6,Alex,Math,0.0
